## DELTA LIVE TABLES - GOLD LAYER

### COACHES DLT PIPELINE

In [0]:
import dlt
from pyspark.sql.functions import *

### Expectations for Data Quality

In [0]:
expec_coaches = {
    "rule1": "code IS NOT NULL",
    "rule2": "current IS TRUE"
}

In [0]:
expec_nocs = {
        "rule1": "code is not null"
    }

In [0]:
expec_events = {
        "rule1": "event is not null"
    }


In [0]:

@dlt.view

def source_coaches():
    df = spark.readStream.table("olympicspariscatalog.silver.coaches")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expec_coaches)

def coaches():
    df = spark.readStream.table("LIVE.source_coaches")
    return df


### NOCS DLT PIPELINE

In [0]:
@dlt.view

def source_nocs():
    df = spark.readStream.table("olympicspariscatalog.silver.nocs")
    return df

In [0]:
@dlt.table
@dlt.expect_all_or_drop(expec_nocs)

def nocs():
    df = spark.readStream.table("LIVE.source_nocs")
    return df

### EVENTS DLT PIPELINE

In [0]:
@dlt.view
def source_events():
    df = spark.readStream.table("olympicspariscatalog.silver.events")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expec_events)

def events():
    df = spark.readStream.table("LIVE.source_events")
    return df

## CDC - APPLY CHANGES (DLT)

In [0]:
# Define the target streaming table
dlt.create_streaming_table(
    "athletes",
    comment="Gold athletes table"
)

In [0]:
@dlt.view
def source_athletes():
    df = spark.readStream.table("olympicspariscatalog.silver.athletes")
    return df


In [0]:
dlt.apply_changes(
    target = "athletes",
    source = "source_athletes",
    keys = ["athlete_id"],
    sequence_by = col("height"),
    stored_as_scd_type = 1
    )